In [105]:
import json_lines
import pandas as pd
import numpy as np
import seaborn as sns
import re
import warnings
warnings.filterwarnings("ignore")
from IPython.core import display as ICD

# Introduction

The following notebook manipulates a large 150,000 entry JSONL data file. It does so by first breaking out dictionaries in the name and address fields fields; it converts the dictionary keys into new dataframe columns. It then calculates metrics for the dataframe, such as % of occupied fields and unique values within columns. I began by first proving all functions and metrics on a subset dataframe with 100 entries. After successfully completing a set of tasks on the small dataframe, I'll scale up to the original dataframe. 

The tasks explored are: 
1. Make a list of all of the nested named fields that appear in any record. Concatenate nested field names using a period '.' to defind named fields for nested records. Present the list in alphabetical order.
2. Answer the following questions for each field in your list from question 1.
    - What percentage of the records contain the field?
    - What are the five most common values of the field?
3. How many distinct first names appear in this data set? 
4. How many distinct street names appear in this data set? 
5. What are the 5 most common US area codes in the phone number field? 
6. What are the 5 most common zip codes


In [2]:
# Load data into list, and then load to dataframe
with json_lines.open('../data/ida_wrangling_exercise_data.2017-02-13.jsonl.gz') as f:
    data = [item for item in f]
    df = pd.DataFrame(data)

### Data exploration 

In [3]:
df.head()

,address,dob,email,id,name,phone,record_date,ssn
0,"{'street': '86314 David Pass Apt. 211', 'city'...",1971-06-30,opark@hotmail.com,01d68a4c598a45559c06f4df0b3d82cb,"{'firstname': 'Cynthia', 'lastname': 'Dawson',...",624-869-4610,2006-07-08T09:02:13,xxx-xx-2412
1,"20722 Coleman Villages\nEast Rose, SC 71064-5894",1965-09-09,sperez@armstrong.com,876ff718291d4397bb1e0477ceee6ad9,"{'firstname': 'Tamara', 'lastname': 'Myers'}",1-594-462-7759,2009-03-28T20:22:57,xxx-xx-8025
2,"{'street': '6676 Young Square', 'city': 'New J...",1993-04-12,uortiz@gmail.com,81753097bf7e4e2085982f422bdb9cda,"{'firstname': 'Jamie', 'lastname': 'Alexander'}",472.218.5065x389,2016-08-30T20:31:39,xxx-xx-0568
3,"0932 Gomez Drives\nLeefort, MD 46879-3166",1977-04-14,palmerdiane@yahoo.com,2c2f7154b80f40ca80d08c5adc54ea45,"{'firstname': 'Angela', 'lastname': 'Garcia', ...",1-663-109-4460x1080,2001-02-15T18:50:35,xxx-xx-9825
4,"{'street': '158 Smith Vista', 'city': 'East Sh...",1970-03-19,nancymaxwell@gmail.com,4f5263f339694d068e17ee7fdbb852b8,"{'firstname': 'Jennifer', 'lastname': 'Rodrigu...",233-423-3823,2014-06-21T14:36:01,xxx-xx-9104


In [4]:
'Number of rows {} '.format(len(df))

'Number of rows 150000 '

This initial dataframe has 150,000 entries. Some of the fields (e.g. address and name) are recorded as dictionaries and strings. Let's start working with a subset (first 100 entries ) of the data frame. Truncating the dataframe will speed up initial code experimentation. 

### Operating on subset of data 

In [333]:
# Subset wtih first 100 entries of dataframe
df_sub = df[0:100].copy()

### 1. Make a list of all of the nested named fields that appear in any record. Concatenate nested field names using a period '.' to defind named fields for nested records. Present the list in alphabetical order.

In [334]:
def expand_dict(dframe, dict_names):
    """This function will take a dataframe and dictionary list as inputs. 
    It will convert dictionary keys into seperate columns within the same dataframe by invoking the apply method.
    It returns a dataframe with concatenated columns"""

    # Creates an empty dataframe
    df_nested = pd.DataFrame()

    # Iterate through list of dictionaries
    for item in dict_names:
        # Expand dictionary keys into columns
        df_nested = pd.concat([df_nested, dframe[item].apply(pd.Series).add_prefix(
            (item + '.'))], axis=1)
        # Drops extra generated column from apply method
        df_nested.drop(item + '.0', axis=1, inplace=True)
        # Add original nested column
        df_nested[item] = dframe[item]
        # Replace nested fields with NaN, otherwise leave string value
        df_nested[item] = df_nested[item].apply(lambda x: np.nan if type(x) == dict else x)

    return df_nested

In [335]:
#Invoke the function above 
df_sub = expand_dict(df_sub, ['address', 'name'])
df_sub.head()

,address.city,address.state,address.street,address.zip,address,name.firstname,name.lastname,name.middlename,name
0,Hoodburgh,RI,86314 David Pass Apt. 211,83973,NaN,Cynthia,Dawson,Claire,NaN
1,NaN,NaN,NaN,NaN,"20722 Coleman Villages\nEast Rose, SC 71064-5894",Tamara,Myers,NaN,NaN
2,New Julie,UT,6676 Young Square,73125,NaN,Jamie,Alexander,NaN,NaN
3,NaN,NaN,NaN,NaN,"0932 Gomez Drives\nLeefort, MD 46879-3166",Angela,Garcia,Alexis,NaN
4,East Sharonstad,ME,158 Smith Vista,42483,NaN,Jennifer,Rodriguez,NaN,NaN


In [336]:
# An alphabetical order list of the fields 
sorted(list(df_sub.columns))

['address',
 'address.city',
 'address.state',
 'address.street',
 'address.zip',
 'name',
 'name.firstname',
 'name.lastname',
 'name.middlename']

### 2. Answer the following questions for each field in your list from question 1.

- What percentage of the records contain the field?
- What are the five most common values of the field?

#### What percentage of the records contain the field?

In [9]:
def col_percent(dframe):
    '''A funciton to loop through columns and return the percentage of populated items.'''
    for item in dframe:
        print('{} = {:.1f}%'.format(item, 100 *
                                    dframe[item].count() / len(dframe)))

In [10]:
# Invoke the function above 
col_percent(df_sub)

address.city = 41.0%
address.state = 41.0%
address.street = 41.0%
address.zip = 41.0%
address = 52.0%
name.firstname = 66.0%
name.lastname = 66.0%
name.middlename = 26.0%
name = 34.0%


#### What are the five most common values of the field?

In [11]:
# Five most common values in a column
def common_values(df, num):
    '''Function takes a dataframe and int as input, returns int number of common values'''
    for item in df:
        # Prints common values as a dataframe
        ICD.display(pd.DataFrame(df[item].value_counts().head(num)) )

In [12]:
common_values(df_sub,5)

,address.city
Courtneyport,1
Sanchezshire,1
Christopherfurt,1
Rileychester,1
West Jonathan,1


,address.state
UT,4
RI,4
WY,3
NE,2
CT,2


,address.street
158 Smith Vista,1
47627 Bender Common Suite 388,1
6676 Young Square,1
17685 Jason Lane Apt. 833,1
93406 Justin Spur,1


,address.zip
56062-1568,1
89500-9726,1
47769,1
97258-4639,1
24700,1


,address
"61039 Jacob Cape\nSouth Daniellebury, OK 08139-3165",1
"1471 Nichols Hill Apt. 029\nClintonport, AL 76610",1
"395 Anna Roads Apt. 613\nAlexandershire, FL 22077",1
"PSC 9738, Box 3367\nAPO AE 85789",1
"577 Brown Skyway Suite 354\nShaneview, VI 02325",1


,name.firstname
Robert,3
Shane,2
Steven,2
Daniel,2
Eric,2


,name.lastname
Davis,3
Garcia,2
White,2
Rodriguez,2
Chapman,2


,name.middlename
David,2
Jennifer,1
Gabrielle,1
Arthur,1
Cynthia,1


,name
Paul Russo,1
Crystal Bennett,1
Casey Haas,1
Bernard Wilkerson,1
Cindy Banks,1


### 3. How many distinct first names appear in this data set? Explain your procedure for identifying distinct first names.

Combined 'name.namefirst' and 'name' column, removed prefixes, then used unique() method.

In [317]:
def first_names(df):
    '''A function to determine the distinct names in a list of columns.  
    First it drops an NaN values, then prefixes are substituted with blank spaces, then split again and first name is selected.'''
    df_out = pd.DataFrame()
    for item in df:
        # Drops and NaN Values
        df = item.dropna()
        # grab the last element in address and remove numbers after dash
        df = df.apply(lambda x: re.sub('Mr.|Dr.|Mrs.', '', x).split()[0])
        df_out = pd.concat([df_out, df])
    return 'There are {} distinct first names'.format(len(df_out[0].unique()))

In [318]:
first_names([df_sub['name.firstname'], df_sub.name])

'There are 80 distinct first names'

### 4. How many distinct street names appear in this data set? Explain your procedure for identifying distinct street names.


In [19]:
def street_names(df):
    '''A function to return the distinct street names. 
    The function takes a dataframe column as input. It removes street, Apt., and Suite numbers.
    Returns an dataframe with top 5 values and prints the number of distinct street names'''
    # Drops and NaN Values
    df = df.dropna()
    # remove numbers, and split at Apt. or Suite instances, only take first part of split
    df = df.apply(lambda x: re.sub("\d+", "", re.split('Apt. | Suite', x)[0]))
    ICD.display(pd.DataFrame(df.value_counts().head(5)))
    return 'There are {} distinct street names'.format(len(df.unique()))

In [20]:
street_names( df_sub['address.street'] )

,address.street
Mcclure Crossroad,1
Mills Estates,1
Barnes Loaf,1
Phillips Tunnel,1
Jordan Mall,1


'There are 41 distinct street names'

### 5. What are the 5 most common US area codes in the phone number field? Explain your approach to identify the US area codes in this data set.

In [31]:
# Copy the first 100 phone entries for testing
df_sub['phone'] = df['phone'][0:100].copy()

In [116]:
def area_codes(df):
    '''A function to return the 5 most common zip codes. It takes a dataframe column as input. 
    First it drops an NaN values, then is removes any extensions, and removes special charactors,
    then slices the area from right with [-7:-4], thus not having to worry about prefixes or country codes'''
    # Drops NaN Values
    df = df.dropna()
    # Split at extensions that start with x keep first element, remove special char, slice out area code from right
    df = df.apply(lambda x: re.sub('\W+', '', re.split('x',  x)[0])[-7:-4])
    return df.value_counts().head(5)

In [117]:
area_codes(df_sub['phone'])

461    2
013    2
782    2
869    2
770    1
Name: phone, dtype: int64

### 6. What are the 5 most common zip codes

In [327]:
def zip_codes(df):
    '''A function to return the 5 most common zip codes. It takes a list of dataframe columns as input. 
    First it drops an NaN values, then it grabs the last element of the value, finally, if there's a dash the value is split there.'''
    df_out = pd.DataFrame()
    for item in df:
        # Drops and NaN Values
        df = item.dropna()
        # grab the last element in address and remove numbers after dash
        df = df.apply(lambda x: re.split('-',  x.split()[-1])[0])
        df_out = pd.concat([df_out,df])
    return df_out[0].value_counts().head(5)

In [328]:
zip_codes([  df_sub['address'],df_sub['address.zip'] ])

45729    1
76610    1
16358    1
71064    1
78860    1
Name: 0, dtype: int64

# Now we can begin to work on the large data set

### 1. Make a list of all of the nested named fields that appear in any record. Concatenate nested field names using a period '.' to defind named fields for nested records. Present the list in alphabetical order.

In [132]:
df_nested = expand_dict(df, ['address', 'name'])
df_nested.head()

,address.city,address.state,address.street,address.zip,address,name.firstname,name.lastname,name.middlename,name
0,Hoodburgh,RI,86314 David Pass Apt. 211,83973,NaN,Cynthia,Dawson,Claire,NaN
1,NaN,NaN,NaN,NaN,"20722 Coleman Villages\nEast Rose, SC 71064-5894",Tamara,Myers,NaN,NaN
2,New Julie,UT,6676 Young Square,73125,NaN,Jamie,Alexander,NaN,NaN
3,NaN,NaN,NaN,NaN,"0932 Gomez Drives\nLeefort, MD 46879-3166",Angela,Garcia,Alexis,NaN
4,East Sharonstad,ME,158 Smith Vista,42483,NaN,Jennifer,Rodriguez,NaN,NaN


In [133]:
# An alphabetical order list of the fields 
sorted(list(df_sub.columns))

['address',
 'address.city',
 'address.state',
 'address.street',
 'address.zip',
 'name',
 'name.firstname',
 'name.lastname',
 'name.middlename']

### 2. Answer the following questions for each field in your list from question 1.

- What percentage of the records contain the field?
- What are the five most common values of the field?

In [135]:
col_percent(df_nested)

address.city = 40.8%
address.state = 40.8%
address.street = 40.8%
address.zip = 40.8%
address = 50.1%
name.firstname = 70.0%
name.lastname = 70.0%
name.middlename = 29.1%
name = 28.7%


In [136]:
common_values(df_nested,5)

,address.city
New Michael,61
Lake Michael,58
East Michael,57
Port Michael,56
North Michael,56


,address.state
NC,1109
DC,1106
MI,1097
MD,1095
OH,1085


,address.street
74634 Mitchell Common Suite 667,1
40269 Carmen Dam,1
86331 Murray Hills,1
62193 Michael Corner,1
16977 Daniel Views,1


,address.zip
50032,5
53097,5
98018,5
26231,5
99528,5


,address
"019 Kenneth Lane Apt. 946\nEast Shelby, PW 11143",1
"8192 Morgan Light\nParkerville, KY 64262",1
"95380 Goodwin Springs Apt. 908\nMarkmouth, SC 93982",1
"878 Shannon Square\nBrownmouth, AK 50228",1
"04351 Gonzalez Circles\nReneetown, NM 51205-6469",1


,name.firstname
Michael,2356
David,1577
James,1551
Jennifer,1543
John,1479


,name.lastname
Smith,2373
Johnson,1784
Williams,1417
Brown,1350
Jones,1296


,name.middlename
Michael,977
David,703
Jennifer,656
James,622
John,589


,name
David Smith,20
John Smith,19
Michael Smith,18
Michael Johnson,17
Christopher Smith,16


### 3. How many distinct first names appear in this data set? 


In [324]:
first_names([df_nested['name.firstname'], df_nested.name])

'There are 693 distinct first names'

### 4. How many distinct street names appear in this data set? Explain your procedure for identifying distinct street names.


In [141]:
street_names( df_nested['address.street'] )

,address.street
Michael Mission,11
Michael Ports,10
Williams Stravenue,9
Michael Square,9
Michael Stream,9


'There are 50780 distinct street names'

### 5. What are the 5 most common US area codes in the phone number field? Explain your approach to identify the US area codes in this data set.

Area code 245 is the most common area in the data set, with 181 instances. 

In [142]:
area_codes(df['phone'])

245    181
493    176
525    175
865    175
960    172
Name: phone, dtype: int64

### 6. What are the 5 most common zip codes

In [143]:
zip_codes([  df_nested['address'],df_nested['address.zip'] ])

08079    10
17241     9
02821     8
79941     8
81581     8
Name: 0, dtype: int64